In [1]:
#Simple ECM optimization case for a given building incidence
#Author Bill Zhai--Nov 14, 2019
from gurobipy import *
import numpy as np
import json

In [2]:
#Parameters
#RUN THIS CODE BLOCK TO LOAD THE CONSTRAINTS!
num_ecm = 82; #number of ecm packages
num_bldgType = 13;
num_vintage = 5;
num_climateZone = 9;
#ECM to building type compatibility matrix 
#B_ij is binary, 1 if ecm package i can be applied to building type j
#0, otherwise

#ECM to ECM conflict matrix
#F_ij is: 1 if ecm package i cannot be used with ecm package j, i != j
#2 if ecm package i can be used with ecm package j, i != j
#2 for diagonal entries
B = [];
F = [];
with open('./Measures.json') as file:
    data = json.load(file)
    for ecm in data:
        #prepare B matrix
        #a binary list of 7 building types this ecm package can be used on
        bldgTypeAllowed = np.zeros(num_bldgType)
        if 'Bldg_type' in data[ecm]:
            bldgTypeList = data[ecm]['Bldg_type'].split(';') 
            for i in bldgTypeList:
                try:
                    bldgTypeAllowed[int(i)-1] = 1
                except: #in case of bad json literal input
                    None
        else:#if 'Bldg_type' field doesn't exist, assume the ecm can be used in all building types!!!
            bldgTypeAllowed = np.ones(num_bldgType)
        bldgTypeAllowed = bldgTypeAllowed.tolist()
        B.append(bldgTypeAllowed)
        
        #prepare F matrix
        conflict = 2 * np.ones(num_ecm)
        if 'conflict_measure_ids' in data[ecm]:
            conflictECMs = data[ecm]['conflict_measure_ids'].split(',')
            for i in conflictECMs:
                try:
                    if i != ecm:#comparing two single char strings
                        conflict[int(i)-1] = 1
                except: #in case of bad json literal input
                    None
            #if no confict measure ids are specified, assume the ecm is
            #not in confict with any other ecms
        conflict = conflict.tolist()
        F.append(conflict)    
    
#energy savings, CO2 reductions and financial costs:
#each of these is a 4-d matrix
#energy saving percentage
S = np.zeros([num_ecm, num_bldgType, num_vintage, num_climateZone])
#CO2 reduction
CR = np.zeros([num_ecm, num_bldgType, num_vintage, num_climateZone])
#financial cost
C = np.zeros([num_ecm, num_bldgType, num_vintage, num_climateZone])
with open('./measure_savings.json') as file:
    data = json.load(file)
    for ecm in data:
        ecmID = int(data[ecm]['measure_id'])
        bldgType = int(data[ecm]['building_type_id'])
        vintageID = int(data[ecm]['vintage_id'])
        climateZone = int(data[ecm]['climate_zone'])
        S[ecmID-1, bldgType-1, vintageID-1, climateZone-1] = data[ecm]['saving_pct']
        CR[ecmID-1, bldgType-1, vintageID-1, climateZone-1] = data[ecm]['co2_reduction_klbs']
        C[ecmID-1, bldgType-1, vintageID-1, climateZone-1] = data[ecm]['cost']
    #convert ndarray to nested lists:
    S = S.tolist();
    CR = CR.tolist();
    C = C.tolist();
    

In [3]:
CR

[[[[0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.6],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 14.4, 0.0, 0.0, 0.0, 0.0, 0.0, 15.840000000000002]],
  [[0.0, 0.0, -20.0, 0.0, 0.0, 0.0, 0.0, 0.0, -22.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, -18.0, 0.0, 0.0, 0.0, 0.0, 0.0, -19.8]],
  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
  [[0.0, 0.0, 956.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1051.6000000000001],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [

In [4]:
#function to optimize ecm package configurations for a given building instance
# NOte: should query baselieEnergy from baseline.json file with bldgType, vintageID, climateZone!!!
# Add Payback year: can be constraint or objective function
#def optimizeECM(bldgType, vintageID, climateZone, objective_term, budget=np.inf, paybackYears=np.inf):
def optimizeECM(bldgType, vintageID, climateZone, baselineEnergy, budget):
    m = Model('ECM_optimization')
    #decision variables: x_i for i=1,...,100 whether to install ecm package i on building
    x = m.addVars([n for n in range(0, num_ecm)], vtype=GRB.BINARY, name='x')

    m.setObjective(sum(x[i]*CR[i][bldgType-1][vintageID-1][climateZone-1] for i in range(0, num_ecm)), GRB.MAXIMIZE)
    #add constraints:
    m.addConstrs(x[i] <= B[i][bldgType-1] for i in range(0, num_ecm))
    m.addConstrs((x[i]+x[j]) <= F[i][j] for i in range(0, num_ecm) for j in range(0, num_ecm) if i != j)
    m.addConstr(sum(C[i][bldgType-1][vintageID-1][climateZone-1]*x[i] for i in range(0, num_ecm)) <= budget)
    #m.addConstr(sum(CR[i-1][bldgType-1][vintageID-1][climateZone-1]*x[i] for i in range(1, num_ecm+1)) >= minCO2ReductionPct)
    #optimize and print results
    
    m.optimize() #later can specify which algorithm to use

    xResults = []
    totalCost = 0
    # Print results
    print ("\nSolution:")
    for v in m.getVars():
        if v.x == 1:
            print('%s: %g' % (v.varName, v.x))
        xResults.append(v.x)
    print('Obj--total saving percentage: %g' % m.objVal)
    for i in range(0,num_ecm):
        totalCost = totalCost + xResults[i]*C[i][bldgType-1][vintageID-1][climateZone-1]

    #get total cost:
    m.getVars()
    print('total cost ')
    print(totalCost)

In [5]:
#Optimize on an example: 
#"building_type_id": 1,
#        "building_type_name": "SmallOffice",
#        "vintage_id": 1,
#        "climate_zone": 3,
#        "energy": 51100
optimizeECM(bldgType = 1, vintageID = 5, climateZone = 3, baselineEnergy = 45990, budget = 10000)

GurobiError: License expired 2020-02-03 - license file '/Users/pengyuanzhai/gurobi.lic'